# Visualizing COVID-19 cases

In [1]:
import altair as alt
import pandas as pd
import io
import requests
from altair import datum
import numpy as np

In [2]:
from altairThemes import standardTheme as st
alt.themes.register('standardTheme', st.standardTheme)
alt.themes.enable('standardTheme')

ThemeRegistry.enable('standardTheme')

## Data Transformation

In [3]:
def case_counts(url):
    s=requests.get(url).content
    cases=pd.read_csv(io.StringIO(s.decode('utf-8')))\
        .drop(['Province/State', 'Lat', 'Long'], axis=1)\
        .groupby(['Country/Region']).sum()\
        .T.reset_index()
    
    cases['index'] = pd.to_datetime(cases['index'])
    cases = cases.melt(id_vars=['index'])
    cases.columns = ['date', 'country', 'value']
    
    return cases

In [4]:
# confirmed cases:
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'

In [5]:
confirmed = case_counts(confirmed_url)
deaths = case_counts(deaths_url)

In [6]:
deaths.describe()

,value
count,8990.000000
mean,15.943159
std,190.439135
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,3405.000000


In [7]:
selCountries = [
    'Germany',
    'Italy',
    'France',
    'United Kingdom',
    'US',
    'Spain',
    'Korea, South'
]

In [8]:
confirmed_sel = confirmed.loc[(confirmed.country.isin(selCountries)&(confirmed.value>=100)),:].copy()
deaths_sel = deaths.loc[(deaths.country.isin(selCountries))&(deaths.value>0),:].copy()

## Case count per day

In [9]:
alt.Chart(
    confirmed_sel, 
    height=300,
    title='Development of COVID-19 cases across countries'
).mark_line(
    point=alt.OverlayMarkDef(shape='circle', size=10)
).encode(
    alt.X('date:T', title='Date'),
    alt.Y(
        'value:Q', 
        scale=alt.Scale(
            type='log', 
            bins=[100,500,1000,5000,10000,50000,100000]),
        title='Confirmed cases (log scale)'
    ),
    color='country:N')

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## Case count since 100th confirmed case in country

In [10]:
confirmed_sel['days_since100'] = 0
for name, group in confirmed_sel.groupby(['country']):
    for i in range(0, len(group)):
        ix = group.index.values[i]
        confirmed_sel.loc[ix, 'days_since100']=i

In [11]:
chart_covid_cases = alt.Chart(
    confirmed_sel, 
    height=300,
    title='Development of COVID-19 cases across countries'
).mark_line(
    point=alt.OverlayMarkDef(shape='circle', size=10)
).encode(
    alt.X('days_since100:Q', title='Days since 100th case'),
    alt.Y(
        'value:Q', 
        scale=alt.Scale(
            type='log', 
            bins=[100,500,1000,5000,10000,50000,100000]),
        title='Confirmed cases (log scale)'
    ),
    color='country:N'        
)

In [12]:
chart_covid_cases

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [15]:
chart_covid_cases.save('./charts/covid_cases.svg', scale_factor=1.5)

## Percentage growth to previous day

In [16]:
confirmed_sel['perc_growth'] = np.nan
for name, group in confirmed_sel.loc[:,['country','value']].groupby('country'):
    for i in range(1,len(group)):
        prev = group.loc[group.index.values[i-1], 'value']
        now = group.loc[group.index.values[i],'value']
        growth = now/prev
        confirmed_sel.loc[group.index.values[i],'perc_growth'] = growth
        

In [17]:
chart_growth_rates= alt.Chart(
    confirmed_sel, 
    height=300,
    title='Growth rates of COVID-19 cases across countries'
).mark_line().encode(
    alt.X('date:T', title='Days since 100th case'),
    alt.Y(
        'perc_growth:Q',
        title='Growth rate compared to previous day'
    ),
    color='country:N')

In [18]:
chart_growth_rates

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [20]:
chart_growth_rates.save('./charts/growth_rates.svg', scale_factor=1.5)

## Cases per 10^6 inhabitants

In [21]:
population = {
    'France':65129728,
    'Germany':83517045,
    'Italy':60550075,
    'Korea, South':51225308,
    'Spain':46736776,
    'US':329064917,
    'United Kingdom': 67530172,
}

In [22]:
confirmed_sel['cases_per1m'] = np.nan
for name, group in confirmed_sel.loc[:,['country','value']].groupby('country'):
    for i in range(0,len(group)):
        cases = group.loc[group.index.values[i],'value']
        cases_per = cases/(population[name]/10**6)
        confirmed_sel.loc[group.index.values[i],'cases_per1m'] = cases_per

In [23]:
chart_cases_per_1m = alt.Chart(
    confirmed_sel, 
    height=300,
    title='Development of confirmed COVID-19 cases per 1m inhabitants'
).mark_line().encode(
    alt.X('days_since100:Q', title='Days since 100th case'),
    alt.Y(
        'cases_per1m:Q',
        title='Cases per 1m inhabitants'
    ),
    color='country:N')

In [24]:
chart_cases_per_1m

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [26]:
chart_cases_per_1m.save('./charts/cases_per_1m.svg', scale_factor=1.5)

## Cases per Intensive Care Units (ICU)

Definitions of ICUs vary a lot between countries. Numbers have to be viewed with care...
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3568952/#B12
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3551445/

In [27]:
icu_per_100000 = {
    'France':np.nan,
    'Germany':25,
    'Italy':np.nan,
    'Korea, South':np.nan,
    'Spain':10,
    'US':np.nan,
    'United Kingdom': 3,
}

In [28]:
confirmed_sel['cases_perICU'] = np.nan
for name, group in confirmed_sel.loc[:,['country','value']].groupby('country'):
    for i in range(0,len(group)):
        cases = group.loc[group.index.values[i],'value']
        cases_per = cases/(icu_per_100000[name]*population[name]/10**5)
        confirmed_sel.loc[group.index.values[i],'cases_perICU'] = cases_per

In [29]:
chart_cases_per_icu = alt.Chart(
    confirmed_sel, 
    height=300,
    title='Development of confirmed COVID-19 cases per ICU'
).mark_line().encode(
    alt.X('days_since100:Q', title='Days since 100th case'),
    alt.Y(
        'cases_perICU:Q',
        title='Cases per ICU'
    ),
    color='country:N')

In [30]:
chart_cases_per_icu

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [32]:
chart_cases_per_icu.save('./charts/cases_per_icu.svg', scale_factor=1.5)

## Deaths

In [33]:
chart_deaths = alt.Chart(
    deaths_sel, 
    height=300,
    title='Development of confirmed COVID-19 deaths'
).mark_line(
    point=alt.OverlayMarkDef(shape='circle', size=10)
).encode(
    alt.X('days_since100:Q', title='Days since 100th case'),
    alt.Y(
        'value:Q',
        title='COVID-19 deaths'
    ),
    color='country:N')

In [34]:
chart_deaths

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [35]:
chart_deaths.save('./charts/deaths.svg', scale_factor=1.5)

## Deaths compared to infections

In [36]:
both = confirmed.join(deaths, rsuffix='_d').drop(['date_d', 'country_d'], axis=1)

In [37]:
both.columns =['date', 'country', 'confirmed', 'deaths']

In [38]:
both['death_rate']=both.deaths/both.confirmed

In [39]:
both_sel = both.loc[(both.country.isin(selCountries))&(both.confirmed>100),:].copy()

In [40]:
chart_deaths_infections = alt.Chart(
    both_sel, 
    height=300,
    title='Development of confirmed COVID-19 deaths in infected persons'
).mark_line(
    point=alt.OverlayMarkDef(shape='circle', size=10)
).encode(
    alt.X('date:T', title='Date'),
    alt.Y(
        'death_rate:Q',
        title='share of COVID-19 deaths in infected persons'
    ),
    color='country:N')

In [41]:
chart_deaths_infections

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [45]:
chart_deaths_infections.save('./charts/deaths_infections.svg', scale_factor=1.5)

In [46]:
both_sel['days_since100'] = 0
for name, group in both_sel.groupby(['country']):
    for i in range(0, len(group)):
        ix = group.index.values[i]
        both_sel.loc[ix, 'days_since100']=i

In [47]:
chart_deaths_infections_100 = alt.Chart(
    both_sel, 
    height=300,
    title='Development of confirmed COVID-19 deaths in infected persons'
).mark_line(
    point=alt.OverlayMarkDef(shape='circle', size=10)
).encode(
    alt.X('days_since100:Q', title='Days since 100th case'),
    alt.Y(
        'death_rate:Q',
        title='share of COVID-19 deaths in infected persons'
    ),
    color='country:N')

In [48]:
chart_deaths_infections_100

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [49]:
chart_deaths_infections_100.save('./charts/deaths_infections_100.svg', scale_factor=1.5)

In [50]:
both_sel.columns

Index(['date', 'country', 'confirmed', 'deaths', 'death_rate',
       'days_since100'],
      dtype='object')

## Exporting the data

In [51]:
deaths_sel.to_csv('./data/deaths_sel.csv', sep=',', index=False)

In [52]:
both_sel.to_csv('./data/all_sel.csv', sep=',', index=False)

In [53]:
confirmed_sel.to_csv('./data/confirmed_sel.csv', sep=',', index=False)